In [ ]:
!pip install faiss-gpu nlu pyspark==3.0.1 kaggle transformers

     |████████████████████████████████| 89.7 MB 18 kB/s 
     |████████████████████████████████| 460 kB 50.5 MB/s 
     |████████████████████████████████| 204.2 MB 40 kB/s 
     |████████████████████████████████| 3.1 MB 27.8 MB/s 
     |████████████████████████████████| 198 kB 50.0 MB/s 
     |████████████████████████████████| 133 kB 44.2 MB/s 
     |████████████████████████████████| 59 kB 6.3 MB/s 
     |████████████████████████████████| 596 kB 30.5 MB/s 
     |████████████████████████████████| 3.3 MB 38.8 MB/s 
     |████████████████████████████████| 895 kB 44.2 MB/s 
  Created wheel for pyspark: filename=pyspark-3.0.1-py2.py3-none-any.whl size=204612246 sha256=69ffa834946eb4dadb2f4961af25132c0484293cf4647312799586d23b441430
  Stored in directory: /root/.cache/pip/wheels/5e/34/fa/b37b5cef503fc5148b478b2495043ba61b079120b7ff379f9b
Successfully built pyspark
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully unin

In [ ]:
!KAGGLE_USERNAME=octohand KAGGLE_KEY= kaggle datasets download residentmario/wikipedia-article-titles -f titles.txt
!unzip /content/titles.txt.zip

In [ ]:
import faiss
import numpy as np
import pandas as pd
import re
from transformers import pipeline, AutoTokenizer
import nlu

In [ ]:
# 1. get wikipedia English article title dataset: https://www.kaggle.com/residentmario/wikipedia-article-titles. filter for single words
# 2. for each word, extract out the ELMO embeddings: https://medium.com/spark-nlp/1-python-line-for-elmo-word-embeddings-with-john-snow-labs-nlu-628e9b924a3
# 3. push the embeddings into a vector DB
# 4. try the "king - man + woman" example

In [ ]:
elmo = nlu.load('elmo')

elmo download started this may take some time.
Approximate size to download 334.1 MB
[OK!]
sentence_detector_dl download started this may take some time.
Approximate size to download 354.6 KB
[OK!]


In [ ]:
elmo.predict("elmo was trained",  get_embeddings=True)

Exception occured
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/nlu/pipe/utils/predict_helper.py", line 103, in predict_help
    drop_irrelevant_cols, return_spark_df,get_embeddings)
  File "/usr/local/lib/python3.7/dist-packages/nlu/pipe/utils/predict_helper.py", line 24, in __predict_standard_spark
    output_level=output_level,
  File "/usr/local/lib/python3.7/dist-packages/nlu/pipe/pipeline.py", line 374, in pythonify_spark_dataframe
    anno_2_ex_config,self.light_pipe_configured, get_embeddings)
  File "/usr/local/lib/python3.7/dist-packages/nlu/pipe/pipeline.py", line 325, in unpack_and_apply_extractors
    return apply_extractors_and_merge(pdf.toPandas().applymap(extract_pyspark_rows), anno_2_ex_config,
  File "/usr/local/lib/python3.7/dist-packages/pyspark/sql/pandas/conversion.py", line 138, in toPandas
    pdf = pd.DataFrame.from_records(self.collect(), columns=self.columns)
  File "/usr/local/lib/python3.7/dist-packages/pyspark/sql/datafr

No accepted Data type or usable columns found or applying the NLU models failed. 
Make sure that the first column you pass to .predict() is the one that nlu should predict on OR rename the column you want to predict on to 'text'  
On try to reset restart Jupyter session and run the setup script again, you might have used too much memory
Full Stacktrace was (<class 'py4j.protocol.Py4JJavaError'>, Py4JJavaError('An error occurred while calling o461.collectToPython.\n', JavaObject id=o469), <traceback object at 0x7f17c491c7d0>)
Additional info:
<class 'py4j.protocol.Py4JJavaError'> predict_helper.py 103
An error occurred while calling o461.collectToPython.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 2.0 failed 1 times, most recent failure: Lost task 0.0 in stage 2.0 (TID 2, a4de18ad4510, executor driver): java.lang.UnsupportedOperationException: sun.misc.Unsafe or java.nio.DirectByteBuffer.<init>(long, int) not available
	at io.netty.util.internal.

Exception: ignored

In [ ]:
df = pd.read_csv('/content/titles.txt', sep='\n')

In [ ]:
df.columns = ['articles']
df.drop_duplicates(inplace=True)
df.articles = df.articles.apply(str)
articles = df.articles.tolist()
articles = list(set(articles))
articles = [re.sub('_', ' ', article) for article in articles]
articles = [re.sub('[^A-Za-z0-9\s]+', '', article) for article in articles]
final_articles = [article for article in articles if article != '']
final_articles = [article.strip() for article in final_articles]
final_articles = [article for article in final_articles if len(article.split())==1]

In [ ]:
final_articles[0:10]

['Broekhuizen',
 'Humavox',
 'Dybowko',
 'Domiciano',
 'Megen',
 'EuFOD',
 'Trumpist',
 'Kaknu',
 'Zebe',
 'BOO']

In [ ]:
# direct encoding of the sample sentence
# tokenizer = AutoTokenizer.from_pretrained('distilroberta-base')
# encoded_seq = tokenizer.encode("i am sentence")

# pipeline approach
feature_extraction = pipeline('feature-extraction', model="distilroberta-base", tokenizer="distilroberta-base")
features = feature_extraction("i am sentence")

# Compare lengths of outputs
# print(len(encoded_seq)) # 5
# Note that the output has a weird list output that requires to index with 0.
print(len(features[0])) # 5
len(features[0][0])

Downloading:   0%|          | 0.00/480 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/316M [00:00<?, ?B/s]

Some weights of the model checkpoint at distilroberta-base were not used when initializing RobertaModel: ['lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

5


768

In [ ]:
sample_embeddings = []
for doc in final_articles[0:10000]:
  sample_embeddings.append(feature_extraction(doc)[0][0])

In [ ]:
len(final_articles)

1829237

In [ ]:
d = 768

In [ ]:
np.array(sample_embeddings).shape

(10000, 768)

In [ ]:
sample_embeddings = np.array(sample_embeddings).astype('float32')

In [ ]:
res = faiss.StandardGpuResources()  # use a single GPU

nlist = 100
quantizer = faiss.IndexFlatL2(d)
# here we specify METRIC_L2, by default it performs inner-product search
index_ivf = faiss.IndexIVFFlat(quantizer, d, nlist, faiss.METRIC_L2)

# make it an IVF GPU index
gpu_index_ivf = faiss.index_cpu_to_gpu(res, 0, index_ivf)

assert not gpu_index_ivf.is_trained
gpu_index_ivf.train(sample_embeddings)
assert gpu_index_ivf.is_trained

# add vectors to the index
gpu_index_ivf.add(sample_embeddings)
print(gpu_index_ivf.ntotal)

10000


In [ ]:
# query the ith element's neighbours
ith = 1000
D, I = gpu_index_ivf.search(np.array([sample_embeddings[ith]]), 5)
print(I[-5:])
print(final_articles[ith], final_articles[I[0][0]], final_articles[I[0][1]], final_articles[I[0][2]], final_articles[I[0][3]], final_articles[I[0][4]])

[[1000 9874 7938  817 6065]]
C6H13SNO3 C6H13SNO3 C19H32O3 C15H12O6 C21H26NO3 C19H29N3O


In [ ]:
# batch process the rest
batch_len = 5000
for i in range(10000, len(final_articles[10000:]), batch_len):
  embeddings = []
  for doc in final_articles[i:i+batch_len]:
    embeddings.append(feature_extraction(doc)[0][0])
  embeddings = np.array(embeddings).astype('float32')
  gpu_index_ivf.add(embeddings)

KeyboardInterrupt: ignored

In [ ]:
d = 64                           # dimension
nb = 100000                      # database size
nq = 10000                       # nb of queries
np.random.seed(1234)             # make reproducible
xb = np.random.random((nb, d)).astype('float32')
xb[:, 0] += np.arange(nb) / 1000.
xq = np.random.random((nq, d)).astype('float32')
xq[:, 0] += np.arange(nq) / 1000.

In [ ]:
res = faiss.StandardGpuResources()  # use a single GPU

## Using a flat index

index_flat = faiss.IndexFlatL2(d)  # build a flat (CPU) index

# make it a flat GPU index
gpu_index_flat = faiss.index_cpu_to_gpu(res, 0, index_flat)

gpu_index_flat.add(xb)         # add vectors to the index
print(gpu_index_flat.ntotal)

k = 4                          # we want to see 4 nearest neighbors
D, I = gpu_index_flat.search(xq, k)  # actual search
print(I[:5])                   # neighbors of the 5 first queries
print(I[-5:])                  # neighbors of the 5 last queries


## Using an IVF index

nlist = 100
quantizer = faiss.IndexFlatL2(d)  # the other index
index_ivf = faiss.IndexIVFFlat(quantizer, d, nlist, faiss.METRIC_L2)
# here we specify METRIC_L2, by default it performs inner-product search

# make it an IVF GPU index
gpu_index_ivf = faiss.index_cpu_to_gpu(res, 0, index_ivf)

assert not gpu_index_ivf.is_trained
gpu_index_ivf.train(xb)        # add vectors to the index
assert gpu_index_ivf.is_trained

gpu_index_ivf.add(xb)          # add vectors to the index
print(gpu_index_ivf.ntotal)

k = 4                          # we want to see 4 nearest neighbors
D, I = gpu_index_ivf.search(xq, k)  # actual search
print(I[:5])                   # neighbors of the 5 first queries
print(I[-5:])                  # neighbors of the 5 last queries


100000
[[ 381  207  210  477]
 [ 526  911  142   72]
 [ 838  527 1290  425]
 [ 196  184  164  359]
 [ 526  377  120  425]]
[[ 9900 10500  9309  9831]
 [11055 10895 10812 11321]
 [11353 11103 10164  9787]
 [10571 10664 10632  9638]
 [ 9628  9554 10036  9582]]
100000
[[ 381  207  210  477]
 [ 526  911  142   72]
 [ 838  527 1290  425]
 [ 196  184  164  359]
 [ 526  377  120  425]]
[[ 9900  9309  9810 10048]
 [11055 10895 10812 11321]
 [11353 10164  9787 10719]
 [10571 10664 10632 10203]
 [ 9628  9554  9582 10304]]
